In [ ]:
!./build.sh

In [ ]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports create_project.py

In [ ]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)

from dotenv import load_dotenv
import json

# Test Lambda

In [ ]:
from create_project import handler
from microkit.utils import collect_cet_now

In [ ]:
payload_dict = {
    'name': 'Test 01', 
    'updated_by': 'pluto',
    "updated_at": collect_cet_now()
}
test_event = {"body": json.dumps(payload_dict)}
response = handler(event=test_event, context=_)

In [ ]:
assert response['statusCode'] == 200 

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [ ]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/create-project.zip"
FUNCTION_NAME = "quack-tsln-create-project"
ENV_DYNAMODB = os.environ["DB_NAME"]
ENV_VARIABLES = {
    'Variables': {
        'BUCKET': BUCKET, 
        'DB_NAME': ENV_DYNAMODB, 
        "LAMBDA_RUNTIME_ENV": "cloud",
    }
}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [ ]:
build_package(PACKAGE)

In [ ]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

In [ ]:
DESC = "Lambda to Create a Project"
MAIN_PYFILE = "create_project"

In [ ]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES,
        code_info=CODE_INFO,
        memory_size=128
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

In [ ]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)

In [ ]:
# Create Query Parameter
# "&".join([f"{key}={value}" for key, value in test_event["queryStringParameters"].items()])